[View in Colaboratory](https://colab.research.google.com/github/selvam-learn/Cat-Dog-Classifier/blob/master/DNN_using_plain_TF_Cat_vs_Dog_classifier_Kaggle_dataset/DNN%20using%20plain%20TF%20-%20Cat%20vs%20Dog%20classifier_Colab%20file.ipynb)

In [ ]:
import tensorflow as tf
import numpy as np
import h5py
import math
import time
from random import shuffle
import matplotlib.pyplot as plt
from tensorflow.contrib.layers import fully_connected, l2_regularizer
from tensorflow.contrib.framework import arg_scope
from sklearn.preprocessing import MinMaxScaler

%matplotlib inline

### Util functions

In [ ]:
def load_mini_dataset(index):
    filename_prefix = 'datasets/dog_vs_cat_normalized_dataset_kaggle_128px_'
    filename = filename_prefix + str(index) + ".h5"
    with h5py.File(filename, "r") as f:
    
        #print(list(f.keys()))
        x_mini_data = None
        y_mini_data = None
        
        x_mini_data = f["input_data"][:]
        y_mini_data = f["input_labels"][:]
        
    return x_mini_data, y_mini_data

In [ ]:
def get_shuffled_indices(start_value, last_value):
    
    temp = np.arange(start_value, last_value)
    shuffle(temp)
    
    return temp

In [ ]:
def create_random_mini_batches(X, Y, n_classes = 2, mini_batch_size = 32):
    
    m = X.shape[0] # number of training examples
    mini_batches = []
    
    #print("Shape of X =", X.shape)
    #print("Shape of Y =", Y.shape)
    
    #Reshaping to convert Y to a 2D array from a rank one array
    Y = Y.reshape(Y.shape[0], 1)
    
    #Shuffle the data in each of the mini batch
    permutation = list(np.random.permutation(m))
    shuffled_X = X[permutation, :]
    shuffled_Y = Y[permutation, :]
    
    n_mini_batches = math.ceil(m / mini_batch_size) # number of mini batches of size mini_batch_size in your partitionning
    for k in range(n_mini_batches):
        
        start_pos = k * mini_batch_size
        end_pos = min(start_pos + mini_batch_size, m)
        
        mini_batch_X = shuffled_X[start_pos : end_pos, :]
        mini_batch_Y = shuffled_Y[start_pos : end_pos, :]
        
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    return mini_batches

### Build the DNN using plain TensorFlow

**Step 1 - Code the individual ops in the computation graph**
- Create Placeholders for X, Y
- Create the nn_model - all the layers and initialize them
- Compute the cost
- Create the optimizer to minimize the cost
- Evaluate the model

**Step 2 - Build the computation graph**
- Combine all the steps in Step 1 to build the computation graph
- Initialize the variables
- Create a Saver object to save the learnt parameters after the model is trained

**Step 3 - Execute the graph**
- Create mini batches so that gradient descent works on these mini batches for every step instead of all instances
- Train the model for a given number of epochs
- Print the cost, train accuracy & test accuracy at regular interval of epochs
- Plot the training error (cost) vs epochs
- Save the parameters

**Step 4 - Predict for any data**
- Build a method to pass random data for the model to predict based on the parameters learnt through training

### Create the placeholders

In [ ]:
def create_placeholders(n_inputs):
    '''
    n_inputs - A scalar containing the number of input features
    '''
    
    X = tf.placeholder(tf.float32, shape = (None, n_inputs), name = 'X')
    Y = tf.placeholder(tf.float32, shape = (None, 1), name = 'Y')
    
    return X, Y

### Create the NN model

In [ ]:
def nn_model(X, n_neurons):
    '''
    X - input Tensor X
    n_neurons - A scalar containing the number of neurons in each layer including both hidden layers and output layer 
    '''
    #with arg_scope([fully_connected], weights_regularizer = l2_regularizer(scale = 0.01)):
    hidden_layer_1 = fully_connected(X, n_neurons['hidden_layer_1'], scope = 'hidden_layer_1')
    hidden_layer_2 = fully_connected(hidden_layer_1, n_neurons['hidden_layer_2'], scope = 'hidden_layer_2')
    hidden_layer_3 = fully_connected(hidden_layer_2, n_neurons['hidden_layer_3'], scope = 'hidden_layer_3')
    hidden_layer_4 = fully_connected(hidden_layer_3, n_neurons['hidden_layer_4'], scope = 'hidden_layer_4')
    hidden_layer_5 = fully_connected(hidden_layer_4, n_neurons['hidden_layer_5'], scope = 'hidden_layer_5')
    logits = fully_connected(hidden_layer_3, n_neurons['output_layer'], activation_fn = None, scope = 'output_layer')
    
    return logits

### Compute the cost

In [ ]:
def compute_cost(labels, logits):
    '''
    labels - label tensor Y
    logits - Tensor containing the values of the output layer before passing to the activation function
    '''
    
    cross_entropy = tf.nn.sigmoid_cross_entropy_with_logits(labels = labels, logits = logits)
    cost = tf.reduce_mean(cross_entropy, name = 'cost')
    #base_cost = tf.reduce_mean(cross_entropy)
    #reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
    #cost = tf.add_n([base_cost] + reg_losses, name = 'cost')
    
    return cost

### Create the optimizer and the training operation

In [ ]:
def optimizer(learning_rate, cost):
    '''
    learning_rate - A scalar value containing the learning rate for the backpropagation step
    cost - Overall cost from the forward propagation step for one set of mini batch instance
    '''
    
    optimizer = tf.train.AdamOptimizer(learning_rate)
    training_op = optimizer.minimize(cost)
    
    return training_op

### Evaluate the model

We will use accuracy as the evaluation metric

In [ ]:
def evaluate_model(logits, Y):
    '''
    logits - An array containing the values from the output layer
    Y - An array containing the labels
    '''
    
    #Compute the probability using the sigmoid function
    y_pred = tf.nn.sigmoid(logits)
    #Convert it to 0 or 1 class based on the probability and cast it to integer
    y_pred = tf.cast(y_pred > 0.5, tf.int64)
    #y_pred = tf.cast(tf.round(y_pred), tf.int64)
    
    #Create a boolean tensor by comparing the model prediction against the labels
    correct_prediction = tf.equal(y_pred, tf.cast(Y, tf.int64))
    #Compute the accuracy across all the instances
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    return accuracy

### Build the computation graph & execute it

In [ ]:
def train_model(n_neurons, n_inputs, learning_rate = 0.01, mini_batch_size = 32, n_epochs = 50, print_cost = False):
    
    tf.reset_default_graph()
    
    #Declare and initialize the required variables
    costs = []
    
    #Create Placeholder
    X, Y = create_placeholders(n_inputs)
    
    #Create the nn_model
    logits = nn_model(X, n_neurons) 
    
    #Compute the cost
    cost = compute_cost(Y, logits)
    
    #Optimize the cost using Gradient Descent Optimizer
    training_op = optimizer(learning_rate, cost)
    
    #Evaluate the model
    accuracy = evaluate_model(logits, Y)
    
    #Initialize the variables
    init = tf.global_variables_initializer()
    
    #Create the Saver object
    saver = tf.train.Saver()
    
    #Execute the Graph - Train the model
    with tf.Session() as sess:
        init.run()
        
        for epoch in range(1, n_epochs + 1):
            n_mini_batches = 0
            epoch_cost = 0
            
            tic = time.time()
            
            file_indices = get_shuffled_indices(1, 26)
            for j in file_indices:
                X_mini = None
                Y_mini = None
                X_mini, Y_mini = load_mini_dataset(j)
                
                mini_batches = None
                mini_batches = create_random_mini_batches(X_mini, Y_mini, mini_batch_size = mini_batch_size)
            
                for mini_batch in mini_batches:
                    n_mini_batches += 1
                    (X_mini_batch, Y_mini_batch) = mini_batch
                    _, mini_batch_cost = sess.run([training_op, cost], feed_dict = {X: X_mini_batch, Y: Y_mini_batch})

                    epoch_cost += mini_batch_cost
            
            toc = time.time()
            print('Epoch', epoch, 'took', ((toc-tic)*1000), 'ms')
            
            epoch_cost = epoch_cost / n_mini_batches
            train_accuracy = accuracy.eval(feed_dict = {X: X_mini_batch, Y: Y_mini_batch})
            #cval_accuracy = accuracy.eval(feed_dict = {X: X_cval, Y: Y_cval})
            
            if print_cost and epoch % 2 == 0:
                print('At epoch', epoch, 'Cost =', epoch_cost, '| Train Accuracy =', train_accuracy)
            
            if epoch % 2 == 0:
                costs.append(epoch_cost)
            
        saver_path = saver.save(sess, './my_model_final.ckpt')
        
    #print('Final - Train Accuracy =', train_accuracy, '| CVal Accuracy =', cval_accuracy)
    print('Final - Train Accuracy =', train_accuracy)
        
    plt.plot(costs)
    plt.xlabel('# of Epochs')
    plt.ylabel('Training Error - Cost')
    plt.title('Training Error Vs Epochs')
    plt.show()
    
    return saver_path

In [ ]:
n_neurons = {'hidden_layer_1': 50,
             'hidden_layer_2': 50,
             'hidden_layer_3': 50,
             'hidden_layer_4': 50,
             'hidden_layer_5': 30,
             'output_layer': 1}
n_input_features = 49152
train_model(n_neurons, n_input_features, learning_rate = 0.01, n_epochs = 200, mini_batch_size = 32,
            print_cost = True)